## Week 4, Lab 2: Predicting Chronic Kidney Disease in Patients
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus on steps exploring data, building models and evaluating the models we build.

There are three links you may find important:
- [A set of chronic kidney disease (CKD) data and other biological factors](./chronic_kidney_disease_full.csv).
- [The CKD data dictionary](./chronic_kidney_disease_header.txt).
- [An article comparing the use of k-nearest neighbors and support vector machines on predicting CKD](./chronic_kidney_disease.pdf).

## Step 1: Define the problem.

Suppose you're working for Mayo Clinic, widely recognized to be the top hospital in the United States. In your work, you've overheard nurses and doctors discuss test results, then arrive at a conclusion as to whether or not someone has developed a particular disease or condition. For example, you might overhear something like:

> **Nurse**: Male 57 year-old patient presents with severe chest pain. FDP _(short for fibrin degradation product)_ was elevated at 13. We did an echo _(echocardiogram)_ and it was inconclusive.

> **Doctor**: What was his interarm BP? _(blood pressure)_

> **Nurse**: Systolic was 140 on the right; 110 on the left.

> **Doctor**: Dammit, it's an aortic dissection! Get to the OR _(operating room)_ now!

> _(intense music playing)_

In this fictitious but [Shonda Rhimes-esque](https://en.wikipedia.org/wiki/Shonda_Rhimes#Grey's_Anatomy,_Private_Practice,_Scandal_and_other_projects_with_ABC) scenario, you might imagine the doctor going through a series of steps like a [flowchart](https://en.wikipedia.org/wiki/Flowchart), or a series of if-this-then-that steps to diagnose a patient. The first steps made the doctor ask what the interarm blood pressure was. Because interarm blood pressure took on the values it took on, the doctor diagnosed the patient with an aortic dissection.

Your goal, as a research biostatistical data scientist at the nation's top hospital, is to develop a medical test that can improve upon our current diagnosis system for [chronic kidney disease (CKD)](https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521).

**Real-world problem**: Develop a medical diagnosis test that is better than our current diagnosis system for CKD.

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

---

## Step 2: Obtain the data.

### 1. Read in the data.

In [1]:
import pandas as pd

data = pd.read_csv('chronic_kidney_disease_full.csv')

In [2]:
data.tail()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,140.0,...,47.0,6700.0,4.9,no,no,no,good,no,no,notckd
396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,75.0,...,54.0,7800.0,6.2,no,no,no,good,no,no,notckd
397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,100.0,...,49.0,6600.0,5.4,no,no,no,good,no,no,notckd
398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,114.0,...,51.0,7200.0,5.9,no,no,no,good,no,no,notckd
399,58.0,80.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,131.0,...,53.0,6800.0,6.1,no,no,no,good,no,no,notckd


In [3]:
data.shape

(400, 25)

In [4]:
data.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'class'],
      dtype='object')

### 2. Check out the data dictionary. What are a few features or relationships you might be interested in checking out?

### Answer:

> I would be interested in checking age, sugar, hypertension, diabetes mellitus, coronary artery disease and red/white blood cell count. 

---

## Step 3: Explore the data.

### 3. How much of the data is missing from each column?

In [5]:
data.isnull().sum()

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

### 4. Suppose that I dropped every row that contained at least one missing value. (In the context of analysis with missing data, we call this a "complete case analysis," because we keep only the complete cases!) How many rows would remain in our dataframe? What are at least two downsides to doing this?

> There's a good visual on slide 15 of [this deck](https://liberalarts.utexas.edu/prc/_files/cs/Missing-Data.pdf) that shows what a complete case analysis looks like if you're interested.

### Answer:

> We would be left with only 158 rows of data, from our original 400. One downside is that you are losing lots of valuable data from the other columns. Also you are decreasing your sample size and making your potential model very vulnerable to inaccuracies

### 5. Thinking critically about how our data were gathered, it's likely that these records were gathered by doctors and nurses. Brainstorm three potential areas (in addition to the missing data we've already discussed) where this data might be inaccurate or imprecise.

### Answer:

> 1. Doctors may have their own biases as to what data they think is important to collect. 

> 2. If this information was collected by doctors and nurses it is likely that people were already in the hospital with known health issues, so the sample seems less random.

> 3. Geographically, this may be a sample size of a specific group of people and may be a less random as well.

---

## Step 4: Model the data.

### 6. Suppose that I want to construct a model where no person who has CKD will ever be told that they do not have CKD. What (very simple, no machine learning needed) model can I create that will never tell a person with CKD that they do not have CKD?

> Hint: Don't think about `statsmodels` or `scikit-learn` here.

### Answer:

> In order to create a model that never told a person with CKD that they do not have CKD, we would need to create a model that told every person they had CKD, regardless of what their biomarkers said. 

### 7. In problem 6, what common classification metric did we optimize for? Did we minimize false positives or negatives?

### Answer:

> We optimized for Sensivity or recall. We minimized false negatives to be zero. 

### 8. Thinking ethically, what is at least one disadvantage to the model you described in problem 6?

### Answer:

> Ethically we are creating fear, panic, further unnecessary testing and potential financial burdens for people who actually do not have CKD

### 9. Suppose that I want to construct a model where a person who does not have CKD will ever be told that they do have CKD. What (very simple, no machine learning needed) model can I create that will accomplish this?

### Answer:

> In order to create a model where a person who does not have CKD will ever be told that they do have CKD, we would need to create a model that told every person they do not have CKD.

### 10. In problem 9, what common classification metric did we optimize for? Did we minimize false positives or negatives?

### Answer:

> We optimaized for Specificity. We minimized false positives

### 11. Thinking ethically, what is at least one disadvantage to the model you described in problem 9?

### Answer: 

> Ethically we are not informing someone that they have a disease when we could potentially be saving their life. 

### 12. Construct a logistic regression model in `sklearn` predicting class from the other variables. You may scale, select/drop, and engineer features as you wish - build a good model! Make sure, however, that you include at least one categorical/dummy feature and at least one quantitative feature.

> Hint: Remember to do a train/test split!

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [7]:
data['class'] = data['class'].map({'ckd': 1, 'notckd': 0})
data.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,1
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,1
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,1
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,1
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,1


In [8]:
data = pd.get_dummies(data)

In [9]:
data.head().T

,0,1,2,3,4
age,48.00,7.00,62.00,48.000,51.00
bp,80.00,50.00,80.00,70.000,80.00
sg,1.02,1.02,1.01,1.005,1.01
al,1.00,4.00,2.00,4.000,2.00
su,0.00,0.00,3.00,0.000,0.00
bgr,121.00,NaN,423.00,117.000,106.00
bu,36.00,18.00,53.00,56.000,26.00
sc,1.20,0.80,1.80,3.800,1.40
sod,NaN,NaN,NaN,111.000,NaN
pot,NaN,NaN,NaN,2.500,NaN


In [10]:
data.drop(columns= ['dm_no',
                    'cad_no',
                    'appet_poor',
                    'pe_no',
                    'ane_no',
                    'rbc_normal',
                    'pc_normal',
                    'pcc_notpresent',
                    'ba_notpresent',
                    'htn_no'], 
          inplace=True)

In [11]:
data.isnull().sum()

age               9
bp               12
sg               47
al               46
su               49
bgr              44
bu               19
sc               17
sod              87
pot              88
hemo             52
pcv              71
wbcc            106
rbcc            131
class             0
rbc_abnormal      0
pc_abnormal       0
pcc_present       0
ba_present        0
htn_yes           0
dm_yes            0
cad_yes           0
appet_good        0
pe_yes            0
ane_yes           0
dtype: int64

In [12]:
data = data.fillna(data.mean())

In [13]:
data.isnull().sum()

age             0
bp              0
sg              0
al              0
su              0
bgr             0
bu              0
sc              0
sod             0
pot             0
hemo            0
pcv             0
wbcc            0
rbcc            0
class           0
rbc_abnormal    0
pc_abnormal     0
pcc_present     0
ba_present      0
htn_yes         0
dm_yes          0
cad_yes         0
appet_good      0
pe_yes          0
ane_yes         0
dtype: int64

In [14]:
# data.dropna(inplace=True)

In [15]:
data.shape

(400, 25)

In [16]:
data['class'].value_counts()

1    250
0    150
Name: class, dtype: int64

In [17]:
#Using list comprehension to get all numeric columns and excluding my y
features = [c for c in data._get_numeric_data().columns if not c in ['class']] 

# Setting my X and my y
X = data[features]
y = data['class']

In [18]:
# Train/test splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [19]:
# Instantiating and fitting standard sclaer
ss = StandardScaler()
ss.fit(X_train);

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


In [20]:
#Scaling the test and train X
X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
# Instantiating and fitting logistic regression
lr = LogisticRegression()
lr.fit(X_train_sc, y_train);

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [22]:
lr.score(X_train_sc, y_train)

1.0

In [23]:
lr.score(X_test_sc, y_test)

0.99

In [24]:
print(f'Logreg predicted values: {lr.predict(X_train.head())}')

Logreg predicted values: [1 1 1 1 1]


In [25]:
pred = lr.predict(X_train_sc)

In [26]:
pred

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1])

---

## Step 5: Evaluate the model.

### 13. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your quantitative features.

In [27]:
print(lr.coef_)


[[-0.05234746  0.48229894 -1.9382426   1.00358306  0.62988692  0.56978356
  -0.26646207  0.63467513 -0.47401419 -0.2614274  -1.39270164 -0.82063757
   0.01122867 -0.67529896  0.27365722  0.21033129 -0.01363558 -0.17503579
   0.44606912  0.4623575  -0.478872   -0.73453706  0.58994226 -0.30213675]]


### Answer:

> My age coefficient is -0.05234746. This means as age increases by one, a person is e^-0.05 times as likely to have CKD

### 14. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your categorical/dummy features.

### Answer: 

> ane_yes coef = -0.30213675. This means if a person has Anemea, that person is e^-.03 times as
likely to have CKD

### 15. Despite being a relatively simple model, logistic regression is very widely used in the real world. Why do you think that's the case? Name at least two advantages to using logistic regression as a modeling technique.

### Answer:

> 1. It is very efficient
> 2. It's a classification algorithm that shares similar properties to linear regression.

### 16. Does it make sense to generate a confusion matrix on our training data or our test data? Why? Generate it on the proper data.

> Hint: Once you've generated your predicted $y$ values and you have your observed $y$ values, then it will be easy to [generate a confusion matrix using sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

In [28]:
X_train.shape

(300, 24)

In [29]:
X_test.shape

(100, 24)

In [30]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train, pred)

In [31]:
cm

array([[115,   0],
       [  0, 185]])

#### 17. In this hospital case, we want to predict CKD. Do we want to optimize for sensitivity, specificity, or something else? Why? (If you don't think there's one clear answer, that's okay! There rarely is. Be sure to defend your conclusion!)

### Answer:

> Ideally we would optimize for both, but if we can only choose one we want to optimize for sensitivity in this case. I think which ever we choose, we want to consider the alternative. In this case if we choose specificity, that would mean that we are making sure we are maximizing our true positives and minimizing our false positives. While on the surface that sounds good (only telling people who actually have CKD that they have the disease) we are then leaving a potential risk for missing those who according to our model are are on the border line and may or may not have the disease. By optimizing for sensitivity, we are casting a wider net and although we may have more false positives, we're also go to have less false negatives where we are telling people they are ok when they actually have the disease.

### 18 (BONUS). Write a function that will create an ROC curve for you, then plot the ROC curve.

Here's a strategy you might consider:
1. In order to even begin, you'll need some fit model. Use your logistic regression model from problem 12.
2. We want to look at all values of your "threshold" - that is, anything where .predict() gives you above your threshold falls in the "positive class," and anything that is below your threshold falls in the "negative class." Start the threshold at 0.
3. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
4. Increment your threshold by some "step." Maybe set your step to be 0.01, or even smaller.
5. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
6. Repeat steps 3 and 4 until you get to the threshold of 1.
7. Plot the values of sensitivity and 1 - specificity.

## No thank you

### 19. Suppose you're speaking with the biostatistics lead at Mayo Clinic, who asks you "Why are unbalanced classes generally a problem? Are they a problem in this particular CKD analysis?" How would you respond?

### Answer:

> If you have a model with an accuracy score of 100% it is actually a terrible model. In our instance we are saying that there is no CKD when we know that to not be true. So if we live and prepare as if there is no CKD, that can actually lead to a lot of CKD going undiagnosed.

### 20. Suppose you're speaking with a doctor at Mayo Clinic who, despite being very smart, doesn't know much about data science or statistics. How would you explain why unbalanced classes are generally a problem to this doctor?

### Answer:

> I would probably tell him the exact same thing. If you have a model with an accuracy score of 100% it is actually a terrible model. In our instance we are saying that there is no CKD when we know that to not be true. So if we live and prepare as if there is no CKD, that can actually lead to a lot of CKD going undiagnosed.

### 21. Let's create very unbalanced classes just for the sake of this example! Generate very unbalanced classes by [bootstrapping](http://stattrek.com/statistics/dictionary.aspx?definition=sampling_with_replacement) (a.k.a. random sampling with replacement) the majority class.

1. The majority class are those individuals with CKD.
2. Generate a random sample of size 200,000 of individuals who have CKD **with replacement**. (Consider setting a random seed for this part!)
3. Create a new dataframe with the original data plus this random sample of data.
4. Now we should have a dataset with around 200,000 observations, of which only about 0.00075% are non-CKD individuals.

In [32]:
import random
random.seed(42)
bs_sample1 = np.random.choice((data['class'] == 1), size=200_000)
bs_sample2 = np.random.choice(data['age'], size=200_000)

In [33]:
bs1_df = pd.DataFrame(bs_sample1)
bs2_df = pd.DataFrame(bs_sample2)

In [34]:
bs1_df.describe()

,0
count,200000
unique,2
top,True
freq,124633


In [35]:
bs2_df.describe()

,0
count,200000.000000
mean,51.463962
std,16.954329
min,2.000000
25%,42.000000
50%,54.000000
75%,64.000000
max,90.000000


In [40]:
bs1_df.rename(columns = {0: 'class'}, inplace=True)
bs2_df.rename(columns = {0: 'age'}, inplace=True)

In [42]:
bs1_df['class'] = bs1_df['class'].map({True: 1, False: 0})

In [43]:
bs1_df.head()

,class
0,0
1,0
2,1
3,0
4,1


In [44]:
bs2_df.head()

,age
0,43.0
1,72.0
2,30.0
3,70.0
4,55.0


In [45]:
ub_data = pd.concat([bs2_df, bs1_df], axis=1)

In [46]:
ub_data.shape

(200000, 2)

In [47]:
ub_data.head()

,age,class
0,43.0,0
1,72.0,0
2,30.0,1
3,70.0,0
4,55.0,1


In [48]:
ub_data = ub_data.fillna(ub_data.median())

In [49]:
ub_data.isnull().sum()

age      0
class    0
dtype: int64

### 22. Build a logistic regression model on the unbalanced class data and evaluate its performance using whatever method(s) you see fit. How would you describe the impact of unbalanced classes on logistic regression as a classifier?
> Be sure to look at how well it performs on non-CKD data.

In [50]:
# Setting my X and my y
X1 = ub_data['age']
y1 = ub_data['class']

In [51]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=42)

In [52]:
ub_lr = LogisticRegression()
ub_lr.fit(X1_train, y1_train);

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: Expected 2D array, got 1D array instead:
array=[56. 45. 49. ... 60. 54. 59.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

---

## Step 6: Answer the problem.

At this step, you would generally answer the problem! In this situation, you would likely present your model to doctors or administrators at the hospital and show how your model results in reduced false positives/false negatives. Next steps would be to find a way to roll this model and its conclusions out across the hospital so that the outcomes of patients with CKD (and without CKD!) can be improved!